# <b>機械学習の基礎-4</b>
EDA、前処理、特徴量エンジニアリング等の機械学習の一連のプロセスについてメモしておく。  


## 機械学習の一連の流れ
大体は下記のような流れで進んでいく。  
必ずしもこれ通りではない。  
EDAとデータクリーニング、前処理（欠損値対応の部分）は通常のデータ分析業務でも大事。    

1. EDA
2. データクリーニング
3. 前処理
4. 特徴量エンジニアリング
5. 特徴量選択
6. ハイパーパラメータチューニング
7. アンサンブル
8. テストデータに対する評価  
   
(3.～6.は適宜繰り返す。単一のモデルに対してだけでなく、複数のモデルに対して行う。)

## EDA(Exploratory data analysis、探索的データ分析)
学習データのデータ構造、統計量の算出、欠損値・外れ値の確認、可視化等を行い、データの分布や性質を理解するプロセス。  
ここで正しくデータを理解できないと正しい評価指標を選択できず、一連の作業が無駄になってしまうので非常に重要。  
いわば機械学習の作業の前提を決める部分。  

データクリーニングもEDAの作業として扱われることが多い。  
ここでは不要なカラム（重複カラム、相関係数が1、欠損が多いカラム）や不要なデータ（重複データ、欠損が多いデータ、外れ値）など、  
明らかに不要な部分や重複部分（意味合い的にも数値的にも重複しているデータ）を学習データから削除する。  

## 前処理(preprocessing)
主に下記の作業が挙げられる。  
* 欠損値対応(データクリーニングの中でやる場合も多い)
* カテゴリカル変数のエンコーディング
* 特徴量スケーリング

### - 欠損値対応
